In [36]:
import glob, os
import pandas as pd
import warnings
import urllib
from urllib.parse import urlencode, quote_plus
import requests
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")

## 18 ~ 20년도 날씨 데이터 가공하기
- 184 : 제주 -> 위도경도 : 33.51411,126.52969 -> 제주시 건입동
- 185 : 고산 -> 위도경도 : 33.29382,126.16283 -> 제주시 한경면
- 188 : 성산 -> 위도경도 : 33.38677,126.8802 -> 제주시 성산읍
- 189 : 서귀포 -> 위도경도 : 33.24616,126.5653 -> 제주시 정방동

In [58]:
input_file = r'C:\Users\김유빈\빅콘\사용한 데이터\2018-2020 날씨데이터' 
data_all = []

data_file_list = glob.glob(os.path.join(input_file,"*"))
for i in data_file_list:
    data = pd.read_csv(i,encoding="cp949")
    data_all.append(data)
climate_data = pd.concat(data_all,axis=0)

In [59]:
climate_data

,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr),합계 일조시간(hr),합계 일사량(MJ/m2)
0,184,2018-01-01,6.4,4.0,2352,8.8,1223,NaN,NaN,NaN,...,9.2,9.3,15.3,18.6,1.4,2.0,NaN,NaN,NaN,NaN
1,184,2018-01-02,7.0,3.0,431,10.1,1520,NaN,NaN,NaN,...,9.2,9.5,16.1,18.7,2.0,2.8,NaN,NaN,NaN,NaN
2,184,2018-01-03,5.0,3.5,2128,8.4,1,NaN,NaN,NaN,...,9.2,9.8,15.9,18.7,1.4,2.0,NaN,NaN,NaN,NaN
3,184,2018-01-04,3.9,2.4,2341,4.6,1149,2.62,NaN,NaN,...,9.4,10.0,16.0,18.7,0.7,1.0,4.8,NaN,NaN,NaN
4,184,2018-01-05,5.1,1.9,54,6.5,1721,8.15,NaN,NaN,...,9.5,10.0,15.5,18.6,1.3,1.9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,189,2020-12-27,10.9,9.9,635,11.9,1249,14.08,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,52.1,NaN,0.0,NaN
362,189,2020-12-28,11.7,7.9,804,15.5,1352,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.2,NaN
363,189,2020-12-29,12.5,9.9,2352,15.2,1239,6.02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,1.4,NaN
364,189,2020-12-30,0.7,-2.1,1907,10.0,1,15.67,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.1,NaN,4.4,NaN


In [60]:
climate_data["지점"].value_counts()

184    1096
188    1096
185    1096
189    1096
Name: 지점, dtype: int64

### 행정동 위도 경도 매핑하기

In [134]:
code = pd.read_csv("행정동코드.csv")
#code["주소"] = code["행정동명"].apply( lambda x : "제주특별자치시 " + x)
#code.to_csv("행정동코드.csv",encoding="utf-8-sig")

## 21년도 날씨데이터 가져오기 - 공공데이터 포털 API
- 인코딩키 : %2FVg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0%2BIYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA%3D%3D
- 디코딩키 : /Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA==

In [42]:
data_table = pd.read_csv("ASOS_데이터정의서.csv")
data_table.head()
#data_table["항목명(영문)"]  = data_table["항목명(영문)"].apply(lambda x : x.strip().lower())

,항목명(영문),항목명(국문)
0,stnid,지점번호
1,stnNm,지점명
2,tm,시간
3,avgTa,평균 기온
4,minTa,최저 기온


In [48]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey
                               , quote_plus('pageNo') : '1'
                               , quote_plus('numOfRows') : '365'
                               , quote_plus('dataType') : 'XML'
                               , quote_plus('dataCd') : 'ASOS'
                               , quote_plus('dateCd') : 'DAY'
                               , quote_plus('startDt') : '20210101'
                               , quote_plus('endDt') : '20210831'
                               , quote_plus('stnIds') : '184' })
open_url_get = open_url + queryParams
res = requests.get(open_url_get)

In [49]:
datas = BeautifulSoup(res.content,"html.parser").find_all("item")
# item : 값에 해당하는 것들만 가져오기

dataframe = []
for data in datas:
    row = []
    for col in data_table["항목명(영문)"]:
        row.append(data.find(col.strip().lower()).get_text())
    dataframe.append(row)
climate_184 = pd.DataFrame(data = dataframe,index=pd.date_range("2021-01-01","2021-08-31"),
                          columns = data_table["항목명(국문)"])

XML에서 태그마다 데이터를 하나씩 가져오는 과정
- 태그를 일일히 다 쓰기 귀찮아서 ASOS 데이터 정의서를 이용
- ASOS 데이터 정의서에 "avgtd"가 "avgtd "로 들어가서 문제발생

관측소 185

In [50]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey
                               , quote_plus('pageNo') : '1'
                               , quote_plus('numOfRows') : '365'
                               , quote_plus('dataType') : 'XML'
                               , quote_plus('dataCd') : 'ASOS'
                               , quote_plus('dateCd') : 'DAY'
                               , quote_plus('startDt') : '20210101'
                               , quote_plus('endDt') : '20210831'
                               , quote_plus('stnIds') : '185' })
open_url_get = open_url + queryParams
res = requests.get(open_url_get)

In [51]:
datas = BeautifulSoup(res.content,"html.parser").find_all("item")
# item : 값에 해당하는 것들만 가져오기

dataframe = []
for data in datas:
    row = []
    for col in data_table["항목명(영문)"]:
        row.append(data.find(col.strip().lower()).get_text())
    dataframe.append(row)
climate_185 = pd.DataFrame(data = dataframe,index=pd.date_range("2021-01-01","2021-08-31"),
                          columns = data_table["항목명(국문)"])

관측소 188

In [52]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey
                               , quote_plus('pageNo') : '1'
                               , quote_plus('numOfRows') : '365'
                               , quote_plus('dataType') : 'XML'
                               , quote_plus('dataCd') : 'ASOS'
                               , quote_plus('dateCd') : 'DAY'
                               , quote_plus('startDt') : '20210101'
                               , quote_plus('endDt') : '20210831'
                               , quote_plus('stnIds') : '188' })
open_url_get = open_url + queryParams
res = requests.get(open_url_get)

In [53]:
datas = BeautifulSoup(res.content,"html.parser").find_all("item")
# item : 값에 해당하는 것들만 가져오기

dataframe = []
for data in datas:
    row = []
    for col in data_table["항목명(영문)"]:
        row.append(data.find(col.strip().lower()).get_text())
    dataframe.append(row)
climate_188 = pd.DataFrame(data = dataframe,index=pd.date_range("2021-01-01","2021-08-31"),
                          columns = data_table["항목명(국문)"])

관측소 189

In [54]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey
                               , quote_plus('pageNo') : '1'
                               , quote_plus('numOfRows') : '365'
                               , quote_plus('dataType') : 'XML'
                               , quote_plus('dataCd') : 'ASOS'
                               , quote_plus('dateCd') : 'DAY'
                               , quote_plus('startDt') : '20210101'
                               , quote_plus('endDt') : '20210831'
                               , quote_plus('stnIds') : '189' })
open_url_get = open_url + queryParams
res = requests.get(open_url_get)

In [55]:
datas = BeautifulSoup(res.content,"html.parser").find_all("item")
# item : 값에 해당하는 것들만 가져오기

dataframe = []
for data in datas:
    row = []
    for col in data_table["항목명(영문)"]:
        row.append(data.find(col.strip().lower()).get_text())
    dataframe.append(row)
climate_189 = pd.DataFrame(data = dataframe,index=pd.date_range("2021-01-01","2021-08-31"),
                          columns = data_table["항목명(국문)"])

2021 날씨데이터와 2020 날씨데이터 열(column) 맞춰주기
- 통일된 날씨데이터 열 : climate.columns
- 2018 ~ 2020 날씨데이터 : climate_data
- 2021 날씨 데이터 : climate

In [56]:
climate = pd.concat([climate_184,climate_185,climate_188,climate_189],axis=0)
climate = climate.iloc[:,:62].drop(["지점명","일기현상","최다 풍향","평균 상대습도 시각"],axis=1).reset_index(drop=True)
climate.to_csv("2021 날씨데이터.csv",
              encoding="utf-8-sig",index=False)

In [61]:
import re
climate_data = climate_data.drop(["최소 상대습도 시각(hhmi)",'합계 일조시간(hr)', '합계 일사량(MJ/m2)'],axis=1)
climate_data_column = []
for i in climate_data.columns:
    climate_data_column.append((re.sub(r'\([^)]*\)', '',i)).strip())

In [62]:
climate_data.columns = climate.columns

### 2018 ~ 2021 날씨 데이터
- all_climate : 2018년부터 2021년 날씨 데이터 합친 것
- 평균기온의 결측기가 존재하는 것들은 전날 다음날의 평균값으로 대체

In [63]:
all_climate = pd.concat([climate_data,climate],axis=0)

In [68]:
all_climate.to_csv("2018~2021 날씨 데이터.csv",index=False,encoding="utf-8-sig")

In [69]:
import pandas as pd
날씨 = pd.read_csv("2018~2021 날씨 데이터.csv")

In [70]:
날씨_기온강수량 = 날씨[['지점번호', '시간', '평균 기온', '최저 기온','최고 기온','일강수량']]

결측치 채우기

In [71]:
for idx,row in 날씨_기온강수량[날씨_기온강수량["평균 기온"].isnull()].iterrows():
    날씨_기온강수량.loc[row.name,"평균 기온"] = ( 날씨_기온강수량.loc[row.name - 1,"평균 기온"] +
                                      날씨_기온강수량.loc[row.name + 1,"평균 기온"] ) /2
    # row.name이 인덱스

In [72]:
날씨_기온강수량

,지점번호,시간,평균 기온,최저 기온,최고 기온,일강수량
0,184,2018-01-01,6.4,4.0,8.8,NaN
1,184,2018-01-02,7.0,3.0,10.1,NaN
2,184,2018-01-03,5.0,3.5,8.4,NaN
3,184,2018-01-04,3.9,2.4,4.6,0.6
4,184,2018-01-05,5.1,1.9,6.5,4.2
...,...,...,...,...,...,...
5351,189,2021-08-27,26.7,25.4,28.9,3.4
5352,189,2021-08-28,26.8,25.3,29.7,NaN
5353,189,2021-08-29,26.7,25.3,29.1,NaN
5354,189,2021-08-30,26.8,25.3,29.3,NaN


### 행정동별로 날씨 매핑하기
- 가까운 관측소의 날씨로 넣어주기

In [73]:
관측소 = [[184,"제주",33.51411,126.52969],
      [185,"고산",33.29382,126.16283],
      [188,"성산",33.38677,126.8802],
      [189,"서귀포",33.24616,126.5653]]

관측소_위경도 = pd.DataFrame(data = 관측소,columns = ["지점번호","관측소","위도","경도"])

In [74]:
날씨_기온강수량_위경도 = pd.merge(날씨_기온강수량,관측소_위경도,on="지점번호",how="left")

In [75]:
날씨_기온강수량_위경도.to_csv("2018~2021 날씨 데이터 전처리완료.csv",index=False,encoding="utf-8-sig")

### 파생변수 만들기
- 월 평균, 표준편차

In [77]:
날씨 = pd.read_csv("2018~2021 날씨 데이터 전처리완료.csv").fillna(0).rename(columns={"시간":"일자"})
날씨["일자"] = pd.to_datetime(날씨["일자"],format="%Y-%m-%d")
날씨["년"] = 날씨["일자"].apply(lambda x : x.year)
날씨["월"] = 날씨["일자"].apply(lambda x : x.month)
날씨["일"] = 날씨["일자"].apply(lambda x : x.day)

In [78]:
날씨.tail(2)

,지점번호,일자,평균 기온,최저 기온,최고 기온,일강수량,관측소,위도,경도,년,월,일
5354,189,2021-08-30,26.8,25.3,29.3,0.0,서귀포,33.24616,126.5653,2021,8,30
5355,189,2021-08-31,26.5,24.1,29.2,0.0,서귀포,33.24616,126.5653,2021,8,31


In [79]:
aggregation = {"평균 기온":["mean","std"],"최저 기온":["mean","std"],
              "최고 기온":["mean","std"],"일강수량":["mean","std"]}

날씨_월별 = 날씨.groupby(["지점번호","월"]).agg(aggregation).reset_index()
날씨_월별.columns = ["지점번호","월","평균기온월평균","평균기온월편차",
                "최고기온월평균","최고기온월편차","최저기온월평균","최저기온월편차","일강수량월평균","일강수량월편차"]

일별 데이터 / 월별 데이터 결합하기

In [80]:
날씨_일별_월별 = pd.merge(날씨,날씨_월별,on=["지점번호","월"],how="inner")

In [88]:
날씨_일별_월별.to_csv("2018~2021 날씨 데이터 일별월별 전처리완료.csv",encoding="utf-8-sig",index=False)

In [87]:
날씨_일별_월별

,지점번호,일자,평균 기온,최저 기온,최고 기온,일강수량,관측소,위도,경도,년,월,일,평균기온월평균,평균기온월편차,최고기온월평균,최고기온월편차,최저기온월평균,최저기온월편차,일강수량월평균,일강수량월편차
0,184,2018-01-01,6.4,4.0,8.8,0.0,제주,33.51411,126.52969,2018,1,1,6.754032,3.459038,4.407258,3.360705,9.267742,3.907124,2.544355,8.351275
1,184,2018-01-02,7.0,3.0,10.1,0.0,제주,33.51411,126.52969,2018,1,2,6.754032,3.459038,4.407258,3.360705,9.267742,3.907124,2.544355,8.351275
2,184,2018-01-03,5.0,3.5,8.4,0.0,제주,33.51411,126.52969,2018,1,3,6.754032,3.459038,4.407258,3.360705,9.267742,3.907124,2.544355,8.351275
3,184,2018-01-04,3.9,2.4,4.6,0.6,제주,33.51411,126.52969,2018,1,4,6.754032,3.459038,4.407258,3.360705,9.267742,3.907124,2.544355,8.351275
4,184,2018-01-05,5.1,1.9,6.5,4.2,제주,33.51411,126.52969,2018,1,5,6.754032,3.459038,4.407258,3.360705,9.267742,3.907124,2.544355,8.351275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5351,189,2020-12-27,10.9,9.9,11.9,53.2,서귀포,33.24616,126.56530,2020,12,27,9.584946,3.621822,6.551613,3.636428,13.043011,3.525324,2.079570,7.163624
5352,189,2020-12-28,11.7,7.9,15.5,0.0,서귀포,33.24616,126.56530,2020,12,28,9.584946,3.621822,6.551613,3.636428,13.043011,3.525324,2.079570,7.163624
5353,189,2020-12-29,12.5,9.9,15.2,4.3,서귀포,33.24616,126.56530,2020,12,29,9.584946,3.621822,6.551613,3.636428,13.043011,3.525324,2.079570,7.163624
5354,189,2020-12-30,0.7,-2.1,10.0,3.9,서귀포,33.24616,126.56530,2020,12,30,9.584946,3.621822,6.551613,3.636428,13.043011,3.525324,2.079570,7.163624
